In [1]:
import json
import numpy as np
import pandas as pd

In [2]:
cards = pd.read_json('cards.json')
cards.drop(columns=['description', 'arena'], inplace=True)
cards = cards.set_index('key')

In [3]:
cards

,name,sc_key,elixir,type,rarity,id
key,,,,,,
knight,Knight,Knight,3,Troop,Common,26000000
archers,Archers,Archer,3,Troop,Common,26000001
goblins,Goblins,Goblins,2,Troop,Common,26000002
giant,Giant,Giant,5,Troop,Rare,26000003
pekka,P.E.K.K.A,Pekka,7,Troop,Epic,26000004
...,...,...,...,...,...,...
earthquake,Earthquake,Earthquake,3,Spell,Rare,28000014
barbarian-barrel,Barbarian Barrel,BarbLog,2,Spell,Epic,28000015
heal-spirit,Heal Spirit,Heal,1,Troop,Rare,28000016


In [4]:
cards.to_csv('data/cards.csv')

In [5]:
with open('cards_stats.json') as file:
    cards_stats_full = json.load(file)
# print(cards_stats_full['characters'][0])

In [6]:
name_to_key = {}
for i in cards_stats_full:
    for j in cards_stats_full[i]:
        if 'key' in j:
            name_to_key[j['name']] = {j['key']}
            if 'summon_character' in j:
                if j['summon_character'] not in name_to_key:
                    name_to_key[j['summon_character']] = {j['key']}
                else:
                    name_to_key[j['summon_character']].add(j['key'])

In [7]:
cards_by_key = {i: {} for i in cards.index.values}
cards_by_name = {}
for i in cards_stats_full:
#     print('---------------------------')
#     print(i)
#     print('------')
#     print(cards_stats_full[i][0].keys())
#     print('---------------------------')
    for j in cards_stats_full[i]:
        name = j['name']
        cards_by_name[name] = j
        
        if name in name_to_key:
            for k in name_to_key[name]:
                if k is not None:
                    cards_by_key[k] = {**j, **cards_by_key[k]}

            
        if 'key' in j:
            if j['key'] in cards_by_key:
                cards_by_key[j['key']] = {**cards_by_key[j['key']] , **j}
            else:
                cards_by_key[j['key']] = j

In [8]:
detail = {}
missing = {}
for i in cards.index.values:
    if len(cards_by_key[i]) != 0:
        detail[i] = cards_by_key[i]
    else:
        detail[i] = None
        missing[i] = None
missing

{'elixir-collector': None,
 'fireball': None,
 'arrows': None,
 'rocket': None,
 'goblin-barrel': None,
 'mirror': None,
 'the-log': None,
 'barbarian-barrel': None,
 'giant-snowball': None}

In [9]:
missing['elixir-collector'] = 'ElixirCollector'
missing['fireball'] = 'FireballSpell'
missing['arrows'] = 'ArrowsSpell'
missing['rocket'] = 'RocketSpell'
missing['goblin-barrel'] = 'GoblinBarrelSpell'
missing['the-log'] = 'LogProjectile'
missing['giant-snowball'] = 'SnowballSpell'
missing['barbarian-barrel'] = 'BarbLogProjectile'

for i in missing:
    if missing[i] is not None:
        detail[i] = cards_by_name[missing[i]]

In [10]:
detail['knight']['attacks_ground']

True

In [11]:
def add_attribute(att):
    cards[att] = np.nan
    for i in cards.index.values:
        if detail[i] is not None and att in detail[i]:
            cards.loc[i, att] = detail[i][att]
    return cards

In [12]:
attributes = {'range', 'attacks_ground', 'attacks_air', 'flying_height', 'hits_ground', 'hits_air', 'aoe_to_ground', 'aoe_to_air'}
for i in attributes:
    add_attribute(i)

In [13]:
cards.fillna(False, inplace=True)
cards['range'] = cards['range'].astype(int)
cards['flying_height'] = cards['flying_height'].astype(int)
cards['damage_air'] = cards['hits_air'] | cards['attacks_air'] | cards['aoe_to_air']
cards['damage_ground'] = cards['hits_ground'] | cards['attacks_ground'] | cards['aoe_to_ground']
cards.drop(columns=['hits_air', 'attacks_air', 'aoe_to_air', 'hits_ground', 'attacks_ground', 'aoe_to_ground'], inplace=True)

In [14]:
cards

,name,sc_key,elixir,type,rarity,id,flying_height,range,damage_air,damage_ground
key,,,,,,,,,,
knight,Knight,Knight,3,Troop,Common,26000000,0,1200,False,True
archers,Archers,Archer,3,Troop,Common,26000001,0,5000,True,True
goblins,Goblins,Goblins,2,Troop,Common,26000002,0,500,False,True
giant,Giant,Giant,5,Troop,Rare,26000003,0,1200,False,True
pekka,P.E.K.K.A,Pekka,7,Troop,Epic,26000004,0,1200,False,True
...,...,...,...,...,...,...,...,...,...,...
earthquake,Earthquake,Earthquake,3,Spell,Rare,28000014,0,0,False,True
barbarian-barrel,Barbarian Barrel,BarbLog,2,Spell,Epic,28000015,0,0,False,True
heal-spirit,Heal Spirit,Heal,1,Troop,Rare,28000016,0,0,True,True


In [15]:
cards.to_csv('data/cards.csv')

In [16]:
import os
for i in os.walk('battles'):
    _, _, files = i

In [17]:
players = []
for filename in files:
    with open(f'battles/{filename}') as file:
        players.append(json.load(file))
len(players)

104

In [18]:
print(players[0][10].keys())

dict_keys(['type', 'battleTime', 'isLadderTournament', 'arena', 'gameMode', 'deckSelection', 'team', 'opponent', 'isHostedMatch'])


In [19]:
print(players[0][10]['team'][0]['cards'][0].keys())

dict_keys(['name', 'id', 'level', 'starLevel', 'maxLevel', 'iconUrls'])


In [20]:
files[0]

'LYCGVUPY.json'

In [21]:
def add_battle(battles, b):
    assert(len(b['team']) == 1)
    assert(len(b['opponent']) == 1)
    team = b['team'][0]
    op = b['opponent'][0]
    tmp = {}
    tmp['p1_tag'] = team['tag']
    tmp['p1_trophy'] = team['startingTrophies']
    for i, c in enumerate(team['cards']):
        tmp[f'p1_card_{i}_id'] = c['id']
        tmp[f'p1_card_{i}_lv'] = c['level']
    tmp['p2_tag'] = op['tag']
    tmp['p2_trophy'] = op['startingTrophies']
    for i, c in enumerate(op['cards']):
        tmp[f'p2_card_{i}_id'] = c['id']
        tmp[f'p2_card_{i}_lv'] = c['level']
    battles.append(tmp)
    

In [22]:
battles = []
types_filter = {'PvP'}
card_info_filter = {'id', 'name', 'level'}
for player in players:
    for battle in player:
        if battle['type'] in types_filter:
            add_battle(battles, battle)
            

In [23]:
battle_df = {i: [] for i in battles[0].keys()}
for i in battles:
    for k in battle_df:
        battle_df[k].append(i[k])
    

In [24]:
battle_df = pd.DataFrame.from_dict(battle_df)

In [25]:
battle_df

,p1_tag,p1_trophy,p1_card_0_id,p1_card_0_lv,p1_card_1_id,p1_card_1_lv,p1_card_2_id,p1_card_2_lv,p1_card_3_id,p1_card_3_lv,...,p2_card_3_id,p2_card_3_lv,p2_card_4_id,p2_card_4_lv,p2_card_5_id,p2_card_5_lv,p2_card_6_id,p2_card_6_lv,p2_card_7_id,p2_card_7_lv
0,#LYCGVUPY,7273,26000036,12,26000042,6,28000000,12,28000008,14,...,26000006,9,26000032,6,26000037,6,28000001,14,26000008,14
1,#LYCGVUPY,7242,26000036,12,26000042,6,28000000,12,28000008,14,...,26000050,6,26000042,6,26000062,6,28000000,12,28000008,14
2,#LYCGVUPY,7269,26000036,12,26000042,6,28000000,12,28000008,14,...,26000015,9,28000015,9,27000009,12,28000012,9,28000007,9
3,#LYCGVUPY,7236,26000036,12,26000042,6,28000000,12,28000008,14,...,28000008,14,26000007,9,28000000,12,26000006,9,28000005,9
4,#LYCGVUPY,7208,26000036,12,26000042,6,28000000,12,28000008,14,...,26000049,14,26000033,6,26000012,9,28000001,14,26000018,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1421,#88828JC9,6251,26000069,4,26000007,9,27000001,12,28000010,6,...,26000043,14,26000042,6,26000083,6,28000002,8,28000008,14
1422,#88828JC9,6225,26000069,4,26000007,9,27000001,12,28000010,6,...,26000021,12,28000011,6,26000014,12,26000000,14,28000000,12
1423,#88828JC9,6253,26000069,4,26000007,9,27000001,12,28000010,6,...,26000042,6,26000055,6,26000041,14,28000001,14,26000046,6
1424,#88828JC9,6224,26000069,4,26000007,9,27000001,12,28000010,6,...,26000026,6,27000003,12,26000084,14,28000011,6,26000025,9


In [26]:
battle_df.to_csv('data/battles.csv')